<!-- TABS -->
# Transfer learning

After obtaining the data, we insert it into the database.

In [ ]:
# <tab: Text-Classification>
datas = [{'txt': d['x'], 'label': d['y']} for d in data]

In [ ]:
# <tab: Image-Classification>
datas = [{'image': d['x'], 'label': d['y']} for d in data]

## Choose features key from feature listener

In [ ]:
# <tab: MongoDB>
input_key = listener.outputs
training_select = select

In [ ]:
# <tab: SQL>
input_key = listener.outputs
training_select = select.outputs(listener.predict_id)

We can find the calculated feature data from the database.

In [ ]:
feature = list(training_select.limit(1).execute())[0][input_key]
feature_size = len(feature)

Define a validation for evaluating the effect after training.

In [ ]:
from superduper import Dataset, Metric, Validation

def acc(x, y):
    return sum([xx == yy for xx, yy in zip(x, y)]) / len(x)


accuracy = Metric(identifier="acc", object=acc)
validation = Validation(
    "transfer_learning_performance",
    key=(input_key, "label"),
    datasets=[
        Dataset(identifier="my-valid", select=training_select.add_fold('valid'))
    ],
    metrics=[accuracy],
)
model.validation = validation

If we execute the apply function, then the model will be added to the database, and because the model has a Trainer, it will perform training tasks.

In [ ]:
db.apply(model)

Get the training metrics

In [ ]:
# <tab: Scikit-Learn>
# Load the model from the database
model = db.load('model', model.identifier)
model.metric_values

In [ ]:
# <tab: Torch>
!pip -q install matplotlib
from matplotlib import pyplot as plt

# Load the model from the database
model = db.load('model', model.identifier)

# Plot the accuracy values
plt.plot(model.trainer.metric_values['my-valid/acc'])
plt.show()